# OMNI High-res

The OMNI high res data only include the IMF, solar wind speed, and proton density time series, which is why we're focussing now on the low res data. 

For the low-res data file, nodata quantities are as follows for each variable:
- 

In the high-res .asc files, the columns corresponding to the above quantities are detailed below, where the number in parentheses is the indexing in python:
- Bx (GSM) : 15 (14) (nodata = 9999.99)
- By (GSM) : 18 (17) (nodata = 9999.99)
- Bz (GSM) : 19 (18) (nodata = 9999.99)
- Flow speed : 22 (21) (nodata = 99999.9) 
- Dynamic pressure : 28 (27) (nodata = 99.99)

Additionally, we'll need the date information:
- Year : 1 (0)
- DOY : 2 (1) (will need to convert to mm-dd
- Hour : 3 (2)
- Minute : 4 (3) (for the high-res data)

#### OMNI high-res
Here I load the data from ascii format supplied in the OMNI 5 minute .asc files. I save the data in an HDF5 file, which is much faster to read and write. 

TO DO: rewrite this with pandas

**Only run this cell if that file doesn't exist yet.** 

In [ ]:
# get data files
files = os.listdir('../OMNI_data/highres/')
files = [file for file in files if '.asc' in file]

# number of files
nfiles = len(files)
# which columns we need (see above)
usecols = [0,1,2,3,14,17,18,21,27]
# array to hold data
data = np.zeros((0,len(usecols)))

for file in files:
    curfile = open(os.path.join('../OMNI_data/',file),'r')
    curdata = np.genfromtxt(curfile,usecols=usecols)
    data = np.concatenate((data,curdata),axis=0)
    
# get python datetimes
t = OMNI2python_date(data[:,0:4])  
# replace OMNI dates with python dates
data = np.concatenate((t,data[:,4:]))
# sort in ascending time
data = data[data[:,0].argsort()]

# create and save hdf5 file
h5f = h5py.File('../OMNI_data/omni_5min_1981-2017_BVP.h5','w')
h5f.create_dataset('B-V-P',data=data)
h5f.close()

Load

In [ ]:
h5f = h5py.File('../OMNI_data/omni_5min_1981-2017_BVP.h5','r')
omni_hr = h5f['B-V-P'][:]
h5f.close()